In [ ]:
"""
I had a idea from work like below for my final projects, I think I will doing solo but if anyone want to work together you are more than welcome to DM me. 

Background: My job is sale directors for steel companies, so we hitting the road really often, when we going out to visit customer, we will visit 3-4 customer at the same time. Everytime I need to find information like what is the sale volume to customer last month, what is the sizes, what is the average prices or previous month prices, I need to open my laptop, get to the ERP and found the information. 

My project: I will made use a local LLMs and Perplexity as my personal assistance. So I will make RAG to access the database, take out the information I need, example like the list of sizes we sold to customers. The material description can be change but can be also repeating multiple times, the diffirences is in the volume wise.


Interface: I think I will use Gradio to do chat interface, then deploy to Hughing Face with address that only me can access 

Extra Credit: It is already a structured data to begin with hahaha. I am thinking about routing agents as tools call.
There will be 3 tools calling:
1/Use PandasQueryEngines to get information of averages prices etc...
2/Use Tools to drawing a charts using data from PandasQuery Engines results
3/Using tools to passed the Queries to Perplexity when I asking something like where is the company, what information it hads. Meaning outside of the RAG Database and most up-to-date data. 

The table will have 8 columns:
Customer name, Item code, Description, Quantity, Unit, Price per unit, Total value, Billing date. 

This 8 columns enough data for me. From customer name, if I want any other information, Perplexity can help me. 

I choose to reduce the complexity of the data in order to 1 single client data including a very few columns but having many many rows instead, this should fit to my need, no need to get a really complex multiple relationship database. 

FYI: The description will be in Vietnamese so Local LLM need to able to handle multi-language.
"""

In [ ]:
"""
!pip install -q pandas
!pip install -q openai
!pip install -q sentence-transformers
!pip install -q llama-index-program-openai
!pip install -q llama-index-llms-openai
!pip install -q llama-index-experimental
!pip install -q llama-index-embeddings-huggingface
!pip install -q llama-index-embeddings-fastembed
!pip install -q llama-index-embeddings-openai
!pip install -q fastembed
!pip install -q xlrd>=2.0.1
!pip install -q openpyxl
"""

In [26]:
import os
import ollama
import pandas as pd
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core import PromptTemplate, StorageContext,load_index_from_storage, VectorStoreIndex,SimpleDirectoryReader, Document
from llama_index.embeddings.fastembed import FastEmbedEmbedding



In [27]:
dotenv_path = find_dotenv()  # Searches up the directory tree
#print(f"Loading dotenv from: {dotenv_path}")
load_dotenv(dotenv_path)
#openai_api_key = os.getenv('OPENAI_API_KEY')
hf_key = os.getenv('HF_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key

In [28]:
Ollama_MODEL = "llama3.1"  # Specifies the Ollama model to use.
Ollama_LLM = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [29]:
Deepseek_MODEL = "deepseek-coder:6.7b"  # Specifies the Ollama model to use.
Deepseek_LLM = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [30]:
def load_excel_files(folder_path):
    all_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(folder_path, filename)
            try:
                df = pd.read_excel(file_path)
                all_data.append(df)
            except Exception as e:
                print(f"Error reading file {filename}: {e}")
                # Handle the error appropriately (e.g., skip the file, raise an exception)
                return None

    if not all_data:
        print("No .xls files found in the specified folder.")
        return None

    try:
      combined_df = pd.concat(all_data, ignore_index=True)
      return combined_df
    except Exception as e:
      print(f"Error combining dataframes: {e}")
      return None

In [31]:
# initialize empty df and load Sale_Data
df = load_excel_files("Sale_Data")
df.head()

Company_Name  Item_Code           Item_Details  \
0  Công Ty TNHH Một Thành Viên Quang Min   50000304          HR 2.3x250x4C   
1  Công Ty TNHH Một Thành Viên Quang Min   50000304           HR 2.3x45x3C   
2  Công Ty TNHH Một Thành Viên Quang Min   50000363  HR 2.0x1114x3000x220T   
3  Công Ty TNHH Một Thành Viên Quang Min   50000030     CRFH 1.5x170x5Coil   
4  Công Ty TNHH Một Thành Viên Quang Min   50000030     CRFH 1.5x193x5Coil   

   Quantity Unit        Value Currency Billing Date  
0    8470.0   KG  116886000.0      VND   2019-02-16  
1    1135.0   KG   15663000.0      VND   2019-02-16  
2   11110.0   KG  154429000.0      VND   2019-02-15  
3    3200.0   KG   47040000.0      VND   2019-02-20  
4    2304.0   KG   33868800.0      VND   2019-02-20

In [32]:
# Get the unique payer names
unique_payers = df['Company_Name'].unique()

# Display the unique payer names
#unique_payers

In [33]:
#query_engine = PandasQueryEngine(df=df, verbose=True) #OpenAI
query_engine = PandasQueryEngine(df=df, llm=Deepseek_LLM, verbose=True)
#query_engine = PandasQueryEngine(df=df, llm=Ollama_LLM, verbose=True)

prompts = query_engine.get_prompts()
#print(prompts["pandas_prompt"].template)

In [34]:
new_prompt = PromptTemplate(
    """\
You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
The dataframe contains data about sale data. 
It have 8 columns: 
Company_Name is the full name of the buyer
Item_Code is the ERP 
Item_Details is the item information including the thickness, width, length of the steel
Quantity is the purchasing quantity,
Unit is the unit of purchasing quantity,
Value is the purchasing value, 
Currency is the unit of purchasing value, 
Billing Date is the date that issues the billing or invoice					

This is the result of `print(df.head())`: {df_str}

Follow these instructions: {instruction_str}
Query: {query_str}

Return the answer from the dataframe with a natural language explanation of the answer.
"Generate *only* Pandas code to answer the question. Do not include any surrounding text or keywords, such as 'python'."
Expression: """
)

query_engine.update_prompts({"pandas_prompt": new_prompt})
prompts = query_engine.get_prompts()
#print(prompts["pandas_prompt"].template)

In [35]:
# Replace predict with create
def _predict(prompt, **kwargs):
    """Helper function to make predict calls using the Ollama API."""
    global Deepseek_MODEL  # Access the global variable
    response = Deepseek_LLM.chat.completions.create(
        model=Deepseek_MODEL,
        messages=[
            {"role": "system", "content": "You are an pandas dataframe expert, user will gave you an query, you will respond with Python executable query code using Pandas. Follow user instruction step by step."},
            {"role": "user", "content": prompt.format(**kwargs)}
        ]
    )
    #print(prompts["pandas_prompt"].template)
    #print(prompt.format(**kwargs))
    return response.choices[0].message.content

# Monkey patch the predict method onto the Ollama_LLM object
Deepseek_LLM.predict = _predict

In [36]:
def check_Query_output(prompt):
    Error_check = False
    Pandas_output = None  # Initialize Pandas_output
    attempt_count = 0  # Initialize attempt counter

    while not Error_check and attempt_count < 5:  # Corrected while loop condition
        Query_Answer = query_engine.query(prompt)
        print(Query_Answer)
        if Query_Answer.metadata and 'raw_pandas_output' in Query_Answer.metadata:
            Pandas_intrusction = Query_Answer.metadata['pandas_instruction_str']
            Pandas_output = Query_Answer.metadata['raw_pandas_output']
            #print(Pandas_output)
            if "error" in Pandas_output.lower():
                prompt = prompt + " " + Pandas_output + " " + Pandas_intrusction
                Error_check = False
                attempt_count += 1  # Increment attempt counter
            else:
                Error_check = True
        else:
            Pandas_output = str(Query_Answer)  # Return the string of the query answer.
            Error_check = True # exit the loop.

    if attempt_count >= 5 and "error" in Pandas_output.lower():
        Pandas_output = None  # Return empty Pandas_output after 5 attempts

    return Pandas_output

In [37]:
# Read documents from the specified directory and load a specific document, "report.pdf".
documents = [Document(text=payer_name) for payer_name in df['Company_Name'].unique()]

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
embeddings = FastEmbedEmbedding()
index = VectorStoreIndex.from_documents(documents, embeddings=embeddings)
                                    
# Create a storage context
storage_context = StorageContext.from_defaults(vector_store=index.vector_store)

# Persist the storage context to disk
storage_context.persist()

# Convert the VectorStoreIndex object into a query engine. This query engine can be used to
# perform semantic searches on the index, matching natural language queries to the most relevant
# documents in the index.
query_engine_name = index.as_query_engine()

# Use the query engine to search for documents that are relevant to the query
# from the indexed documents based on the semantic understanding of the query.
#response = query_engine_name.query("What is company asaba full name?")

# Print the response obtained from the query. This will display the result of the semantic search,
# showing the information or documents that best match the query about the 2024 outlook.
#print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [38]:
def get_company_name(query):
    response = Ollama_LLM.chat.completions.create(
        model=Ollama_MODEL,
        messages=[{"role": "user", "content": f"Extract the company name from this query: '{query}'. Return only the company name."}]
    )
    return response.choices[0].message.content
#get_company_name("What did company asaba buy?")

In [39]:
def update_company_name(query_text, company, response):

    updated_query = query_text.lower().replace(company.lower(), response)
    return updated_query

In [40]:
def update_query_with_company_name(query_text):

    #print(f"Original Query: {query_text}")

    company = get_company_name(query_text)
    #print(f"Extracted Company: {company}")

    response = query_engine_name.query(f"What is the company:'{company}' full name?")
    #print(f"Search Response: {response}")

    response_text = str(response)

    updated_query = update_company_name(query_text, company, response_text)
    #print(f"Updated Query: {updated_query}")

    return updated_query

In [128]:
def process_query(message):
    updated_query = update_query_with_company_name(message)
    result = check_Query_output(updated_query)
    return result

In [129]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import PydanticSingleSelector
from llama_index.core.tools import FunctionTool

In [139]:
process_query_tool = FunctionTool.from_defaults(
    fn=process_query, 
    description="Useful for getting information from the database"
)

In [140]:
get_full_name_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine_name,
    description="Useful for getting full name of company from the database",
)

In [141]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import ReActAgent
from llama_index.core.memory import ChatMemoryBuffer

# Example LLM
llm = OpenAI()

# Manually provide context_window
context_window_size = 4000  # Replace with the actual context window size of your Ollama model
token_limit = int(context_window_size * 0.8) #Adjust the ratio as needed.

agent = ReActAgent.from_tools(
    tools=[process_query_tool, get_full_name_tool],
    llm=llm,
    verbose=True,
    memory_cls=ChatMemoryBuffer,
    memory_kwargs={"token_limit": token_limit}
)

In [142]:
def agent_chat(message, history):
    response = agent.chat(message)
    return response.response

In [143]:
# Create the chat interface for the agent
iface_agent = gr.ChatInterface(
    fn=agent_chat,
    chatbot=gr.Chatbot(height=300, type="messages"),
    textbox=gr.Textbox(placeholder="Enter your query here..."),
    title="LlamaIndex Agent Chat",
    description="Interact with the ReAct agent.",
    type="messages"
)

# Launch the agent interface
iface_agent.launch()

* Running on local URL:  http://127.0.0.1:7869
INFO:httpx:HTTP Request: GET http://127.0.0.1:7869/gradio_api/startup-events "HTTP/1.1 200 OK"
HTTP Request: GET http://127.0.0.1:7869/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7869/ "HTTP/1.1 200 OK"
HTTP Request: HEAD http://127.0.0.1:7869/ "HTTP/1.1 200 OK"

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
> Running step 7409b3b1-96c8-4aeb-b745-6e56b82f4664. Step input: what is katzden full name?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Thought: I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'Katzden'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Observation: Katzden is a part of the company name "CÔNG TY TNHH KATZDEN VIỆT NAM."
> Running step f2afd3e4-59a3-45a9-b376-97e52a76

Traceback (most recent call last):
  File "/Volumes/MacExtend/opt/anaconda3/envs/llms/lib/python3.12/site-packages/llama_index/experimental/query_engine/pandas/output_parser.py", line 44, in default_output_processor
    safe_exec(ast.unparse(module), {}, local_vars)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Volumes/MacExtend/opt/anaconda3/envs/llms/lib/python3.12/site-packages/llama_index/experimental/exec_utils.py", line 171, in safe_exec
    return exec(__source, _get_restricted_globals(__globals), __locals)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'python' is not defined


INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
```python
df[df['Company_Name'] == 'CÔNG TY TNHH KATZDEN VIỆT NAM']['Item_Details'].tolist()
```

```
> Pandas Output: ['GI 3.0x1200x2440', 'GI 3.0x1200x2440', 'GI 3.0x1200x2440', 'ZAM 1.6 x 1220 x 2440 x 160PCS', 'ZAM 1.6 x 1220 x 2440 x 160PCS', 'ZAM 1.6 x 1220 x 2440 x 160PCS', 'ZAM 1.6 x 1220 x 2440 x 160PCS', 'SECC 1.0*1219*1090*400T', 'SECC 1.0*1219*1090*400T', 'SECC 1.0*1219*1090*400T', 'SUS304 HL 1.0x1000x2000', 'SUS304 2B 3.0x1249x2000', 'EG 1.2x1250x2440x35 TAM', 'EG 1.6x1219x2440x27TAM', 'PO 1.6x1219x2440x54TAM', 'PO 1.6x1219x2440x54TAM', 'PO 3.2x1220x2440x150T', 'Po 1.6x1220x2440x60T', 'Po 1.6x1220x2440x60T', 'po 1.6x1220x2440x180T', 'po 1.6x1220x2440x180T', 'po 1.6x1220x2440x180T', 'po 1.6x1220x2440x180T', 'PO 3.2x1220x2440x150T', 'Po 1.6x1220x2440x60T', 'Po 1.6x1220x2440x60T', 